In [57]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [58]:
files_list = ['2024-09-13_Invest_nv_SUEK.xlsx','2024-09-13_Invest_nv_EUROCHEM.xlsx']
old_data_SU = pd.read_excel(Output_path+files_list[0])
old_data_ECH = pd.read_excel(Output_path+files_list[1])
old_data = pd.concat([old_data_SU,old_data_ECH]).reset_index(drop=True)

In [59]:
new_data = export_from_RISKCUSTOM(f'''select * 
from RISKACCESS."unifiedOcp"
where "sourceTable" = 'sapBwInvestmentReport'
and "reportDate" = TO_DATE('31.08.24', 'DD.MM.YY')
--fetch first 10 rows only''')
new_data.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(36743, 40)

In [60]:
data_OCP_work = new_data[['holding','buCode','salesUnitsId','ocpSegment','counterpartySegment','counterpartyCode','effectiveCurrency','volumeEffectiveCurrency','maturityDateTo']]
data_OCP_work.maturityDateTo = data_OCP_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
data_OCP_work.maturityDateTo = pd.to_datetime(data_OCP_work.maturityDateTo)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_13880\2119538208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.maturityDateTo = data_OCP_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_13880\2119538208.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.maturityDateTo = pd.to_datetime(data_OCP_work.maturityDateTo)


In [61]:
# merge
data_OCP_work.counterpartyCode = data_OCP_work.counterpartyCode.astype(str)
old_data.Контрагент = old_data.Контрагент.astype(str)
data_OCP_work.ocpSegment = data_OCP_work.ocpSegment.fillna('External')

leaft_list = ['holding','Comp_segment','Comp_code','Контрагент','currency','КалендГод/Месяц']
ver_cols = ['Версия_035', 'Версия_045','Версия_075', 'Версия_085']
right_list = ['holding','ocpSegment','buCode','counterpartyCode','effectiveCurrency','maturityDateTo']

old_data['Amount_abs'] = old_data['Amount'].abs()
old_data_pivot = old_data.groupby(as_index=False,dropna=False, by=leaft_list)[['Amount_abs']+ver_cols].agg(sum)
data_OCP_work['Amount_abs'] = data_OCP_work['volumeEffectiveCurrency'].abs()
data_OCP_work_pivot = data_OCP_work.groupby(as_index=False,dropna=False, by=right_list)['Amount_abs'].agg(sum)
# merge
merge_data = old_data_pivot.merge(data_OCP_work_pivot, how='outer', left_on=leaft_list, right_on=right_list, suffixes=['_old','_OCP'])
# merge_data[merge_data.Amount_abs_old!=merge_data.Amount_abs_OCP]

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_13880\3146694621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.counterpartyCode = data_OCP_work.counterpartyCode.astype(str)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_13880\3146694621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.ocpSegment = data_OCP_work.ocpSegment.fillna('External')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_13880\3146694621.py:11: FutureWarning: The provided callable <built-in fun

In [62]:
old_data.columns

Index(['holding', 'Comp_segment', 'Comp_code', 'Завод', 'ПЕ', 'СПП-элемент',
       'Контрагент', 'Cpty_segment', 'Вид проекта', 'Профиль проекта',
       'Проект', 'Вид затрат', 'КалендГод/Месяц', 'Календарный год',
       'Календарный месяц', 'currency', 'Версия_035', 'Версия_045',
       'Версия_075', 'Версия_085', 'Amount', 'Coef_to_USD', 'Amount_in_USD',
       'Amount_abs'],
      dtype='object')

In [63]:
merge_data = is_approximately_equal_for_cols(merge_data,'Amount_abs_old','Amount_abs_OCP')

In [64]:
merge_data[merge_data['Diff__more_0.4']==True]

,holding,Comp_segment,Comp_code,Контрагент,currency,КалендГод/Месяц,Amount_abs_old,Версия_035,Версия_045,Версия_075,Версия_085,ocpSegment,buCode,counterpartyCode,effectiveCurrency,maturityDateTo,Amount_abs_OCP,Diff__more_0.4
4,EUROCHEM,RUFERT,9C00,1000014955,RUB,2024-09-01,0.73,5.468080e+08,0.97,0.83,7636364.37,RUFERT,9C00,1000014955,RUB,2024-09-01,7636364.370,True
5,EUROCHEM,RUFERT,9C00,1000014955,RUB,2024-10-01,0.00,2.100000e+07,0.00,0.00,7636363.64,RUFERT,9C00,1000014955,RUB,2024-10-01,7636363.640,True
6,EUROCHEM,RUFERT,9C00,1000014955,RUB,2024-11-01,0.00,2.100000e+07,0.00,0.00,7636363.64,RUFERT,9C00,1000014955,RUB,2024-11-01,7636363.640,True
7,EUROCHEM,RUFERT,9C00,1000014955,RUB,2024-12-01,1370236.90,2.271280e+07,1644284.28,1370236.90,9349159.74,RUFERT,9C00,1000014955,RUB,2024-12-01,9280647.920,True
8,EUROCHEM,RUFERT,9C00,1000014955,RUB,2025-01-01,0.00,0.000000e+00,0.00,0.00,7636363.64,RUFERT,9C00,1000014955,RUB,2025-01-01,7636363.640,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11632,SUEK,SUEK RU,6700,1000161539,RUB,2024-09-01,0.01,1.261333e+07,0.01,0.03,12613333.35,SUEK RU,6700,1000161539,RUB,2024-09-01,3324166.680,True
11633,SUEK,SUEK RU,6700,1000161898,CNY,2024-09-01,1485000.00,1.485000e+06,1782000.00,1485000.00,1485000.00,SUEK RU,6700,1000161898,CNY,2024-09-01,178200.000,True
11649,SUEK,SUEK RU,6700,2000094138,RUB,2024-09-01,93592.00,9.359200e+04,442627.88,93592.00,93592.00,SUEK RU,6700,2000094138,RUB,2024-09-01,426835.000,True
11705,SUEK,SUEK RU,6A00,1000154618,CNY,2024-09-01,0.00,1.752155e+06,0.00,0.00,1752155.04,SUEK RU,6A00,1000154618,CNY,2024-09-01,175215.504,True


In [66]:
# to excel
Output_file = f'{str(date.today())}_ivest_vs_OCP.xlsx'
with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
    merge_data[merge_data['Diff__more_0.4']==True].to_excel(writer, index=False, sheet_name='compare')
    old_data.to_excel(writer, index=False, sheet_name='data_script')
    new_data.to_excel(writer, index=False, sheet_name='data_unOCP')
